In [1]:
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns

In [ ]:
data = pd.read_csv("date_alegeri_turul2.csv")

varsta_mean, varsta_std = data['Varsta'].mean(), data['Varsta'].std()
venit_mean, venit_std = data['Venit'].mean(), data['Venit'].std()

data['Varsta'] = (data['Varsta'] - data['Varsta'].mean()) / data['Varsta'].std()
data['Venit'] = (data['Venit'] - data['Venit'].mean()) / data['Venit'].std()

x_n = ['Varsta', 'Sex', 'Educatie', 'Venit']
x_1 = data[x_n].values
y_data = np.array(data['Vot'])

with pm.Model() as model1:
  beta_0 = pm.Normal('beta_0', mu=0, sigma=10)
  beta = pm.Normal('beta', mu=0, sigma=10, shape=len(x_n))
  mu = beta_0 + pm.math.dot(x_1, beta)

  theta = pm.Deterministic('theta', pm.math.sigmoid(mu))
  y1 = pm.Bernoulli('y1', p=theta, observed=y_data)

  idata_1 = pm.sample(2000, return_inferencedata=True)

pm.plot_trace(idata_1)
plt.show()
az.plot_forest(idata_1, var_names=["beta"], combined=True, hdi_prob=0.94)
plt.title("Intervale de credibilitate pentru coeficienți (94% HDI)")
plt.show()


Intrucat valorile beta[2] si beta[3] corespunzatoare atributelor nivel de educatie si venit lunar au coeficientii cei mai mari, acestea influenteaza cel mai mult rezultatul

In [ ]:

x_n_2 = ['Educatie', 'Venit']
x_2 = data[x_n_2].values


with pm.Model() as model2:
  beta_0_2 = pm.Normal('beta_0', mu=0, sigma=10)
  beta_2 = pm.Normal('beta', mu=0, sigma=10, shape=len(x_n_2))
  mu = beta_0_2 + pm.math.dot(x_2, beta_2)

  theta = pm.Deterministic('theta', pm.math.sigmoid(mu))
  bd = pm.Deterministic('bd', -beta_0_2/beta[1] - beta_2[0]/beta_2[1] * x_2[:,0])

  y2 = pm.Bernoulli('y2', p=theta, observed=y_data)
  idata_2 = pm.sample(2000, return_inferencedata=True)

posterior_2 = idata_2.posterior.stack(samples=("chain", "draw"))
theta = posterior_2['theta'].mean("samples")

idx = np.argsort(x_2[:,0])
bd = idata_2.posterior['bd'].mean(("chain", "draw"))[idx]
plt.vlines(posterior_2['bd'].mean(), 0, 1, color='pink', label="frontiera de decizie medie")

bd_hpd = az.hdi(posterior_2['bd'].values)
plt.fill_betweenx([0, 1], bd_hpd[0], bd_hpd[1], color='purple', alpha=0.5, label="HDI frontiera de decizie")


In [ ]:
pm.compute_log_likelihood(idata_1,model=model1)
pm.compute_log_likelihood(idata_2,model=model2)
cmp_df = az.compare({'model1':idata_1, 'model2':idata_2},method='BB-pseudo-BMA', ic="waic", scale="deviance")

cmp_df

In [ ]:
irisi = pd.read_csv("iris.csv")
